In [1]:
import pandas as pd
import datetime as datetime
import time
import pycurl, certifi, json
from pandas.io.json import json_normalize
from io import BytesIO

In [2]:
df = pd.read_csv('311cleaned.csv')

In [3]:
df.head()

,REQUEST_ID,CREATED_ON,REQUEST_TYPE,REQUEST_ORIGIN,STATUS,DEPARTMENT,NEIGHBORHOOD,COUNCIL_DISTRICT,WARD,TRACT,PUBLIC_WORKS_DIVISION,PLI_DIVISION,POLICE_ZONE,FIRE_ZONE,X,Y,GEO_ACCURACY
0,205290,2017-12-31T23:17:00,Snow/Ice removal,Website,1,DPW - Street Maintenance,Marshall-Shadeland,1.0,27.0,4.200327e+10,1.0,27.0,1.0,1-14,-80.034743,40.472661,EXACT
1,205289,2017-12-31T22:01:00,Snow/Ice removal,Report2Gov iOS,1,DPW - Street Maintenance,Stanton Heights,9.0,10.0,4.200310e+10,2.0,10.0,5.0,3-7,-79.936661,40.480244,EXACT
2,205288,2017-12-31T21:55:00,Snow/Ice removal,Website,1,DPW - Street Maintenance,New Homestead,5.0,31.0,4.200331e+10,3.0,31.0,4.0,4-20,-79.927033,40.389386,EXACT
3,205287,2017-12-31T18:23:00,Missed Pick Up,Website,1,DPW - Refuse,Mount Washington,2.0,19.0,4.200319e+10,5.0,19.0,3.0,4-27,-80.014213,40.426213,APPROXIMATE
4,205286,2017-12-31T17:56:00,Snow/Ice removal,Report2Gov iOS,1,DPW - Street Maintenance,Stanton Heights,9.0,10.0,4.200310e+10,2.0,10.0,5.0,3-7,-79.937035,40.480705,EXACT


In [4]:
df.describe()

,REQUEST_ID,STATUS,COUNCIL_DISTRICT,WARD,TRACT,PUBLIC_WORKS_DIVISION,PLI_DIVISION,POLICE_ZONE,X,Y
count,79852.000000,79852.000000,76069.000000,75965.000000,7.612800e+04,75966.000000,75965.000000,75922.000000,76784.000000,76784.000000
mean,164666.873704,1.176514,4.882370,16.369157,4.200323e+10,3.029619,16.369157,3.550815,-79.289558,40.095406
std,23625.934393,0.703213,2.571621,8.025851,1.887250e+05,1.461813,8.025851,1.607305,7.360487,3.722107
min,123268.000000,0.000000,1.000000,0.000000,4.200301e+10,0.000000,0.000000,1.000000,-80.088105,0.000000
25%,144463.750000,1.000000,3.000000,10.000000,4.200311e+10,2.000000,10.000000,2.000000,-80.007287,40.421424
50%,164863.500000,1.000000,5.000000,16.000000,4.200318e+10,3.000000,16.000000,4.000000,-79.977465,40.444844
75%,185093.250000,1.000000,7.000000,22.000000,4.200328e+10,5.000000,22.000000,5.000000,-79.934367,40.461995
max,205290.000000,3.000000,9.000000,32.000000,4.200398e+10,6.000000,32.000000,6.000000,0.000000,40.526962


In [5]:
cols = ['REQUEST_ID']
dfwx = pd.DataFrame()  #new dataframe to store results

In [6]:
# https://darksky.net/dev/docs#time-machine-request
dsKey = "4702af8fe8916c618732be6bc6d731f6/";
dsUrl = "https://api.darksky.net/forecast/";
exclude = "?exclude=minutely,hourly,flags";
# dsAPI =  dsUrl + dsKey + dsLat + ',' + dsLon + ',' + Time + exclude
i=0 #$1 x 10000 api calls

In [7]:
for index, row in df.iterrows():
    date = datetime.datetime.strptime(row['CREATED_ON'], '%Y-%m-%dT%H:%M:%S') 
    date -= datetime.timedelta(seconds=14400)
    timestamp = int(date.timestamp())
    dsAPI =  dsUrl + dsKey + str(row['Y']) + ',' + str(row['X']) + ',' + str(timestamp) + exclude
    
    buffer = BytesIO()
    c = pycurl.Curl()
    c.setopt(c.URL, dsAPI)
    c.setopt(c.VERBOSE, True)
    c.setopt(c.CAINFO, certifi.where())
    c.setopt(c.WRITEDATA, buffer)
    c.perform()
    c.close()     
    wx = json.loads(buffer.getvalue().decode('iso-8859-1'))
    buffer.close()
    
    #https://www.kaggle.com/jboysen/quick-tutorial-flatten-nested-json-in-pandas
    #wx = json_normalize(wx, record_path='currently', record_prefix='current', errors='raise') #doesn't work
    wxd = json_normalize(wx, record_path=[['daily', 'data']], meta=['latitude', 'longitude'], record_prefix='d_')
    wxc = wx["currently"]
    wxc = json_normalize(wxc)
    
    #new_row = pd.read_json(wxc, orient='records')
    new_row = pd.concat([pd.Series(row['REQUEST_ID'], name='REQUEST_ID'), wxc, wxd], axis=1)
    
    print (dsAPI, new_row)

    dfwx = pd.concat([dfwx, new_row], axis=0, sort=False)
    
    if i > 3: 
        break
    else: 
        i = i + 1

https://api.darksky.net/forecast/4702af8fe8916c618732be6bc6d731f6/40.4726605,-80.0347434,1514765820?exclude=minutely,hourly,flags    REQUEST_ID  apparentTemperature  cloudCover  dewPoint  humidity  \
0      205290                  9.1        0.74      2.92      0.75   

                  icon  precipIntensity  precipProbability  pressure  \
0  partly-cloudy-night                0                  0   1031.49   

         summary    ...          d_time  d_uvIndex  d_uvIndexTime  \
0  Mostly Cloudy    ...      1514696400          1     1514732400   

   d_visibility  d_windBearing  d_windGust  d_windGustTime  d_windSpeed  \
0          9.09            264        5.32      1514696400         0.57   

    latitude  longitude  
0  40.472661 -80.034743  

[1 rows x 58 columns]
https://api.darksky.net/forecast/4702af8fe8916c618732be6bc6d731f6/40.480244,-79.936661,1514761260?exclude=minutely,hourly,flags    REQUEST_ID  apparentTemperature  cloudCover  dewPoint  humidity  \
0      205289        

In [8]:
dfwx.describe()

,REQUEST_ID,apparentTemperature,cloudCover,dewPoint,humidity,precipIntensity,precipProbability,pressure,temperature,time,...,d_time,d_uvIndex,d_uvIndexTime,d_visibility,d_windBearing,d_windGust,d_windGustTime,d_windSpeed,latitude,longitude
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000e+00,...,5.000000e+00,5.0,5.000000e+00,5.000000,5.000000,5.000000,5.000000e+00,5.000000,5.000000,5.000000
mean,205288.000000,11.896000,0.780000,4.688000,0.722000,0.000340,0.016000,1030.936000,11.896000,1.514757e+09,...,1.514696e+09,1.0,1.514732e+09,9.072000,262.600000,5.318000,1.514696e+09,0.556000,40.449842,-79.969937
std,1.581139,2.673468,0.123491,2.012354,0.019235,0.000498,0.023022,0.528375,2.673468,8.615340e+03,...,0.000000e+00,0.0,0.000000e+00,0.016432,3.361547,0.048683,0.000000e+00,0.047749,0.040653,0.050475
min,205286.000000,9.100000,0.710000,2.920000,0.700000,0.000000,0.000000,1030.350000,9.100000,1.514747e+09,...,1.514696e+09,1.0,1.514732e+09,9.060000,257.000000,5.270000,1.514696e+09,0.510000,40.389386,-80.034743
25%,205287.000000,10.160000,0.720000,3.240000,0.710000,0.000000,0.000000,1030.390000,10.160000,1.514748e+09,...,1.514696e+09,1.0,1.514732e+09,9.060000,262.000000,5.270000,1.514696e+09,0.520000,40.426213,-80.014213
50%,205288.000000,10.710000,0.730000,3.520000,0.720000,0.000000,0.000000,1031.200000,10.710000,1.514761e+09,...,1.514696e+09,1.0,1.514732e+09,9.060000,264.000000,5.320000,1.514696e+09,0.550000,40.472661,-79.937035
75%,205289.000000,14.710000,0.740000,6.850000,0.730000,0.000600,0.030000,1031.250000,14.710000,1.514761e+09,...,1.514696e+09,1.0,1.514732e+09,9.090000,265.000000,5.350000,1.514696e+09,0.570000,40.480244,-79.936661
max,205290.000000,14.800000,1.000000,6.910000,0.750000,0.001100,0.050000,1031.490000,14.800000,1.514766e+09,...,1.514696e+09,1.0,1.514732e+09,9.090000,265.000000,5.380000,1.514696e+09,0.630000,40.480705,-79.927033


In [9]:
dfwx.sample(5)

,REQUEST_ID,apparentTemperature,cloudCover,dewPoint,humidity,icon,precipIntensity,precipProbability,pressure,summary,...,d_uvIndex,d_uvIndexTime,d_visibility,d_windBearing,d_windGust,d_windGustTime,d_windSpeed,latitude,longitude,precipType
0,205288,10.71,0.72,3.52,0.72,partly-cloudy-night,0.0000,0.00,1031.20,Mostly Cloudy,...,1,1514732400,9.06,257,5.38,1514696400,0.63,40.389386,-79.927033,NaN
0,205286,14.71,1.00,6.91,0.71,cloudy,0.0006,0.03,1030.35,Overcast,...,1,1514732400,9.06,265,5.27,1514696400,0.52,40.480705,-79.937035,snow
0,205289,10.16,0.71,3.24,0.73,partly-cloudy-night,0.0000,0.00,1031.25,Mostly Cloudy,...,1,1514732400,9.06,265,5.27,1514696400,0.51,40.480244,-79.936661,NaN
0,205287,14.80,0.73,6.85,0.70,partly-cloudy-day,0.0011,0.05,1030.39,Mostly Cloudy,...,1,1514732400,9.09,262,5.35,1514696400,0.55,40.426213,-80.014213,snow
0,205290,9.10,0.74,2.92,0.75,partly-cloudy-night,0.0000,0.00,1031.49,Mostly Cloudy,...,1,1514732400,9.09,264,5.32,1514696400,0.57,40.472661,-80.034743,NaN


In [10]:
with open('311weather.csv', 'w') as fh:
    dfwx.to_csv(fh, index=False)